In [1]:
# Generate Multiline Vector from Labelbox Labels

from glob import glob
import pandas as pd
import json
import rasterio as rio
from shapely.geometry import LineString
from shapely.geometry import Polygon
from geopandas import GeoDataFrame
import geopandas as gpd

# Extract Chips

import gdal
import os

# Create NDVI

import numpy as np


# Exporting Intersecting Chips

from osgeo import gdal, osr, ogr
from shutil import copyfile

In [2]:
def convert_rasters(src_dir, dest_dir, epsg_format='EPSG:3257', windows=False):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""

    src_dir = add_trailing_slash(src_dir)
    dest_dir = add_trailing_slash(dest_dir)
    
    # If the output folder doesn't exist, create it
    create_dir(dest_dir)

    input_files = glob(src_dir + '/*.tif')
    # Keep track of how many files were converted
    n = 1
    total = len(input_files)
    
    for f in input_files:
        print(f'processing file {n} of {total}', end="\r", flush=True)
        
        n += 1
        
        # The way we've set it up, we save each product into a numbered folder,
        # depending on which layer it's in. To keep this structure, we need to
        # pull out the folder number from the file path.
        # How exactly to do this depends on if you're using Windows or not,
        # since the path conventions are different.
        if windows:
            filename = f.split('\\')[-1]
        else:
            filename = f.split('/')[-1]
            
        # If the respective grouping folders are not available 
        create_dir(dest_dir)
        
        output_filepath = dest_dir + filename


#         Finally, we convert
        converted = gdal.Warp(output_filepath, [f],format='GTiff',
                              dstSRS=epsg_format, resampleAlg='near')
        converted = None
        
    print('Finished')

https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html

In [1]:
import rasterio
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT

In [10]:
import numpy as np
from rasterio.windows import Window


def get_windows(img_dim, patch_size=(240, 240), stride=(240, 240)):
    patch_size = np.array(patch_size)
    stride = np.array(stride)
    img_dim = np.array(img_dim)
    # to take into account edges, add additional blocks around right side edge and bottom edge of raster
    new_img_dim = [img_dim[0] + stride[0],img_dim[1] + stride[0]]
    
    max_dim = (new_img_dim//patch_size)*patch_size - patch_size

    ys = np.arange(0, img_dim[0], stride[0])
    xs = np.arange(0, img_dim[1], stride[1])

    tlc = np.array(np.meshgrid(ys, xs)).T.reshape(-1, 2)
    tlc = tlc[tlc[:, 0] <= max_dim[0]]
    tlc = tlc[tlc[:, 1] <= max_dim[1]]
    
    windows = []
    for y,x in tlc.astype(int):
        windows.append(Window(x, y, patch_size[1], patch_size[0]))

    return windows

In [11]:
window = get_windows((1000, 1000))[0]
window

Window(col_off=0, row_off=0, width=240, height=240)

In [12]:
type(window)

rasterio.windows.Window

In [13]:
dst_window = vrt.window(10, 15, 20, 10)

type(dst_window)

rasterio.windows.Window

In [2]:
granule_loc = 'D:/canopy_data/testing/1056_full_congo_export_v12_all_bands_Feb_11_12_27_20_2021.tif'

In [17]:


with rasterio.open(granule_loc) as src:
    with WarpedVRT(src, crs='EPSG:3257',
                   resampling=Resampling.nearest) as vrt:
        
        data = vrt.read()

In [3]:
src = rasterio.open(granule_loc)

vrt = WarpedVRT(src, crs='EPSG:3257', resampling=Resampling.nearest)

In [5]:
data = vrt.read([1,3])

In [6]:
data

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint16)

In [7]:
data.shape

(2, 4372, 4360)

In [8]:
type(data)

numpy.ndarray

In [11]:
np.count_nonzero(data)

22324254

In [12]:
vrt.shape

(4372, 4360)

In [14]:
help(WarpedVRT)

Help on class WarpedVRT in module rasterio.vrt:

class WarpedVRT(rasterio._warp.WarpedVRTReaderBase, rasterio.windows.WindowMethodsMixin, rasterio.transform.TransformMethodsMixin)
 |  A virtual warped dataset.
 |  
 |  Abstracts the details of raster warping and allows access to data
 |  that is reprojected when read.
 |  
 |  This class is backed by an in-memory GDAL VRTWarpedDataset VRT file.
 |  
 |  Parameters
 |  ----------
 |  src_dataset : dataset object
 |      The warp source.
 |  src_crs : CRS or str, optional
 |      Overrides the coordinate reference system of `src_dataset`.
 |  src_transfrom : Affine, optional
 |      Overrides the transform of `src_dataset`.
 |  src_nodata : float, optional
 |      Overrides the nodata value of `src_dataset`, which is the
 |      default.
 |  crs : CRS or str, optional
 |      The coordinate reference system at the end of the warp
 |      operation.  Default: the crs of `src_dataset`. dst_crs is
 |      a deprecated alias for this paramet